In [1]:
import numpy as np
import pandas as pd
import time
import regex as re
from random import randint
from numpy import linalg as LA
from collections import Counter

In [2]:
# Read in data
with open('../pa5train.txt') as f:
    train = f.readlines()
train = [x.strip() for x in train] 

with open('../pa5test.txt') as f:
    test = f.readlines()
test = [x.strip() for x in test] 

with open('../pa5dictionary.txt') as f:
    dictionary = f.readlines()
dictionary = [x.strip() for x in dictionary] 

In [3]:
# create empty lists for data sorting
trainData = []
trainLabel = []

testData = []
testLabel = []

In [4]:
# Remove spaces and split into label/data
for i in range (0,len(train)):
    train[i] = train[i].split(" ")
    trainLabel.append(train[i][4003])
    trainData.append(train[i][:4003])

for i in range (0,len(test)):
    test[i] = test[i].split(" ")
    testLabel.append(test[i][4003])
    testData.append(test[i][:4003])

In [5]:
# convert label string to int
trainLabel = [int(i) for i in trainLabel]

testLabel = [int(i) for i in testLabel]

In [6]:
# convert data string to int
for i in range(0,len(trainData)):
    trainData[i] = [int(j) for j in trainData[i]]
    
for i in range(0,len(testData)):
    testData[i] = [int(j) for j in testData[i]]

In [25]:
# given weights of each data point, number of features(words), training data, and training label
def calcBestH(Dweight,numWords,data,label):
    # find the best weak learner out of C
    hPos = []
    hNeg = []
    
    # for each word x_i there are 2 classifiers total C = 2*numWords
    for i in range (0,numWords):
        ErrPos = 0
        ErrNeg = 0
    
        # for each data point x
        for j in range (0,len(data)):
        
            # check if h(x_i)+ is true where h is 1 if i is in email
            if data[j][i] == 1:
                HxP = 1
            # h is -1 if i is NOT in email
            else:
                HxP = -1
        
            # check if h(x_i)- is true where h is 1 if i is NOT in email
            if data[j][i] == 0:
                HxN = 1
            else:
                HxN = -1
        
            # if h(x_i)+ != y is TRUE, P = 1
            if HxP != label[j]:
                # add weight * 1
                ErrPos = ErrPos + Dweight[j]
            
            # if h(x_i)- != y is TRUE, P = 1
            if HxN != trainLabel[j]:
                # add weight * 1
                ErrNeg = ErrNeg + Dweight[j]
            
        # add error to the dictionary for each word x_i
        hPos.append((i,ErrPos))
        hNeg.append((i,ErrNeg))

    lowErrPos = sorted(hPos, key=lambda x: x[1])
    lowErrNeg = sorted(hNeg, key=lambda x: x[1])

    if(lowErrPos[0][1] < lowErrNeg[0][1]):
        bestH = (lowErrPos[0][0],lowErrPos[0][1],1)
    else: 
        bestH = (lowErrNeg[0][0],lowErrNeg[0][1],0)
    
    return bestH

In [8]:
def calcErr(data, label, hClass):
    preds = []
    for i in range (0,len(data)):
        # if word x_i is the same as the pos/neg classifier
        if(data[i][hClass[0]] == hClass[2]):
            preds.append(1)
        else:
            preds.append(-1)
        
    wrong = 0
    for i in range (0,len(preds)):
        if(preds[i] != label[i]):
            wrong += 1

    return (wrong/len(preds), preds)

In [9]:
def calcAlpha(errT):
    return (np.log((1-errT)/errT))*0.5

In [10]:
def calcNewD(weights,numDataPt,label,predictions,alpha):
    newW = []
    for i in range(0,numDataPt):
        # weight is D_t*e^(-alpha*y_i*h_t(x_i))
        newW.append(weights[i]*np.exp(-alpha*label[i]*predictions[i]))
    
    Z = sum(newW)
    
    # assign the new weights and normalize them
    for j in range(0,numDataPt):
        weights[j] = newW[j]/Z

In [40]:
# calculate H(x_i) given weak learner H
def calcH(hfunc,data):

    newPreds = []
    # for all data points
    for i in range (0,len(data)):
        # if the word matches the pos/neg
        if data[i][hfunc[0]] == hfunc[2]:
            newPreds.append(1)
        else:
            newPreds.append(-1)
    return newPreds

In [46]:
# get number of training data points and words in dictionary
numDataPt = len(trainData)
numWords = len(dictionary)
# each data i has a origin weight of 1/n or 1/number of training data
Dweight = [1/(numDataPt)]*numDataPt

In [47]:
# for all t calculate new weights and the best classifier
allBestH = []
allAlphas = []
for t in range (0,10):
    # compute best H classifier
    best = calcBestH(Dweight,numWords,trainData,trainLabel)
    
    # calculate predictions based on best H
    trainPred = calcH(best,trainData)
    
    # calc alpha
    a = calcAlpha(best[1])
    
    # recompute d weights
    calcNewD(Dweight,numDataPt,trainLabel,trainPred,a)
    
    # append all values
    allBestH.append(best)
    allAlphas.append(a)

In [48]:
allAlphas

[1.0986122886681098,
 0.89444088470022198,
 0.75083264478255618,
 0.75023321505626261,
 0.53981967991915103,
 0.72440679664352636,
 0.6013225031780155,
 0.64528471161174794,
 0.53880141194664477,
 0.52665554140209581]

In [56]:
trainErr = []
testErr = []
alphaH = []
# for all classifiers
for i in range (0,len(allAlphas)):
    # calculate alpha*h_t(x)
    curH = calcH(allBestH[i],trainData)
    
    curH = [allAlphas[i]*x for x in curH]
    alphaH.append(curH) 

In [69]:
finalH = []
finalH.append(alphaH[0])
# for all T 
for j in range (1,len(allAlphas)):
    curH = []
    
    # for each data point in data set
    for i in range (0,len(trainData)):
        curH.append(finalH[j-1][i]+alphaH[j][i])
    
    finalH.append(curH)    
            

In [77]:
wrong = 0
for i in range (0,len(finalH[3])):
    if(np.sign(finalH[3][i]) != trainLabel[i]):
        wrong += 1

wrong/len(finalH[3])

0.051111111111111114

In [41]:
pred1 = calcH(allBestH[0],trainData)

In [27]:
# for all t calculate new weights and the best classifier
allBestH = []
trainErr = []
testErr = []
for t in range (0,10):
    # compute best H classifier
    best = calcBestH(Dweight,numWords,trainData,trainLabel)
    # clac training err/train predictions
    errTrain, trainPred = calcErr(trainData, trainLabel, best)
    # clac testing err/test predictions
    errTest, testPred = calcErr(testData, testLabel, best)
    
    # calc alpha
    a = calcAlpha(best[1])
    
    # recompute d weights
    calcNewD(Dweight,numDataPt,trainLabel,trainPred,a)
    
    # append all values
    allBestH.append(best)
    trainErr.append(errTrain)
    testErr.append(errTest)

In [28]:
trainErr

[0.1,
 0.24,
 0.11777777777777777,
 0.24,
 0.11777777777777777,
 0.3022222222222222,
 0.11777777777777777,
 0.41333333333333333,
 0.11555555555555555,
 0.5311111111111111]

In [74]:
best = calcBestH(Dweight,numWords,trainData,trainLabel)

In [19]:
four = allBestH[3]

In [29]:
allBestH

[(3018, 0.1, 0),
 (2001, 0.14320987654320938, 1),
 (1423, 0.18217728311636641, 0),
 (3783, 0.18235596771428195, 1),
 (2339, 0.25357427053155118, 0),
 (2089, 0.19018422231703097, 1),
 (620, 0.23100501880669561, 0),
 (1328, 0.21575644562873181, 1),
 (3899, 0.2539599279796379, 0),
 (886, 0.25858978910429703, 1)]

In [23]:
err1, trainPred = calcErr(trainData, trainLabel, allBestH[0])

In [24]:
err1

0.1

In [79]:
alpha1 = calcAlpha(best[1])

In [80]:
alpha1

1.0986122886681098

In [86]:
calcNewD(Dweight,numDataPt,trainLabel,trainPred,alpha1)

In [66]:
newW[448]/Z

0.001235245676640129

In [87]:
Dweight

[0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.011111111111111089,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.011111111111111089,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.011111111111111089,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.011111111111111089,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.0012345679012345651,
 0.00123456790123456

In [55]:
Dweight

[0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222222222,
 0.0022222222222

In [29]:
preds = []
for i in range (0,len(testData)):
    # if word x_i is the same as the pos/neg classifier
    if(testData[i][bestH[0]] == bestH[2]):
        preds.append(1)
    else:
        preds.append(-1)
        
wrong = 0
for i in range (0,len(preds)):
    if(preds[i] != trainLabel[i]):
        wrong += 1

wrong/len(preds)

In [12]:
# get number of training data points and words in dictionary
numDataPt = len(trainData)
numWords = len(dictionary)
# each data i has a origin weight of 1/n or 1/number of training data
Dweight = [1/(numDataPt)]*numDataPt

# find the best weak learner out of C
hPos = []
hNeg = []
# for each word x_i there are 2 classifiers total C = 2*numWords
for i in range (0,numWords):
    ErrPos = 0
    ErrNeg = 0
    
    # for each data point x
    for j in range (0,len(trainData)):
        
        # check if h(x_i)+ is true
        if trainData[j][i] > 0:
            xValP = 1
        else:
            xValP = -1
        
        # check if h(x_i)- is true
        if trainData[j][i] == 0:
            xValN = 1
        else:
            xValN = -1
        
        # if h(x_i)+ != y is TRUE, P = 1
        if xValP != trainLabel[j]:
            # add weight * 1
            ErrPos = ErrPos + Dweight[j]
            
        # if h(x_i)- != y is TRUE, P = 1
        if xValN != trainLabel[j]:
            # add weight * 1
            ErrNeg = ErrNeg + Dweight[j]
            
    # add error to the dictionary for each word x_i
    hPos.append((i,ErrPos))
    hNeg.append((i,ErrNeg))

lowErrPos = sorted(hPos, key=lambda x: x[1])
lowErrNeg = sorted(hNeg, key=lambda x: x[1])

if(lowErrPos[0][1] < lowErrNeg[0][1]):
    bestH = lowErrPos[0]
else: 
    bestH = lowErrNeg[0]

In [15]:
bestH

(3018, 0.1)

In [108]:
thir = [x[13] for x in trainData]

In [112]:
too = list(zip(thir,trainLabel))

In [113]:
too

[(0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (0, 1),
 (0, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (0, 1),
 (1, -1),
 (0, 1),
 (0, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (0, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (1, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, -1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0

In [95]:
hPos

[(0, 0.9977777777777819),
 (1, 0.9888888888888929),
 (2, 0.9600000000000039),
 (3, 0.9911111111111152),
 (4, 0.9044444444444479),
 (5, 0.9911111111111152),
 (6, 0.9911111111111152),
 (7, 0.9933333333333374),
 (8, 0.6977777777777799),
 (9, 0.971111111111115),
 (10, 0.848888888888892),
 (11, 0.9200000000000036),
 (12, 0.9822222222222262),
 (13, 1.000000000000004),
 (14, 0.9888888888888929),
 (15, 0.8177777777777807),
 (16, 0.9888888888888929),
 (17, 0.8688888888888922),
 (18, 0.9488888888888927),
 (19, 0.9377777777777815),
 (20, 0.9111111111111146),
 (21, 0.980000000000004),
 (22, 0.9444444444444482),
 (23, 0.9688888888888928),
 (24, 0.9911111111111152),
 (25, 0.8822222222222256),
 (26, 0.9911111111111152),
 (27, 0.9933333333333374),
 (28, 0.9977777777777819),
 (29, 0.9888888888888929),
 (30, 0.9822222222222262),
 (31, 0.9822222222222262),
 (32, 0.9933333333333374),
 (33, 0.980000000000004),
 (34, 0.9955555555555596),
 (35, 0.9866666666666707),
 (36, 0.9777777777777817),
 (37, 0.86444444